In [1]:
%%time

import joblib, time, os, copy, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchvision

import torch.cuda as cutorch

#enable it when running in google cloud, to upload/download file from/to VM to/from google cloud bucket
from google.cloud import storage

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#turn on cpu for troubleshooting as gpu doesnt throw proper error message, runing in cpu shows more specific error message
#device = torch.device("cpu") 
print(device)
for i in range(cutorch.device_count()):
    print(torch.cuda.get_device_properties("cuda:" + str(i)))

bucket_root_path = "weicheng30417"
project_data_folder = "data/breakfast-img-data/"
torch.manual_seed(0)

cuda:0
_CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)
CPU times: user 516 ms, sys: 396 ms, total: 912 ms
Wall time: 2.5 s


In [2]:
def download_all_data_from_bucket():
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    file_id = project_data_folder
    blobs=bucket.list_blobs(prefix=file_id)
    for blob in blobs:        
        file_names = blob.name.split("/")
        if len(file_names) == 3:
            if file_names[2] != "":
                file_name = file_names[2]            
                blob.download_to_filename(file_name)
                print("Download from {0} to local {1}".format(blob.name, file_name))
                

def download_file_from_bucket(file):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    butcketFile = project_data_folder + file
    blob = bucket.blob(butcketFile)
    blob.download_to_filename(file)
    print("Download from {0} to local {1}".format(butcketFile, file))
    
def upload_all_files():    
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    
    directory = os.getcwd()
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv") or filename.endswith(".ipynb") or filename.endswith(".model"):             
            butcketFile = project_data_folder + file
            blob = bucket.blob(butcketFile)
            blob.upload_from_filename(filename)
            print("Upload from local {0} to {1}".format(filename, butcketFile))
            
def upload_files(files):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    for file in files:
        butcketFile = project_data_folder + file
        blob = bucket.blob(butcketFile)
        blob.upload_from_filename(file)
        print("Upload from local {0} to {1}".format(file, butcketFile))

def flatten(ls):
     for item in ls:
            for child in list(item):
                yield child

def get_segment_positions(x):
    ps = x.segment.split()
    ls = []
    for i in range(len(ps)-1):
        if i == 0:
            ls.append([int(ps[i]), int(ps[i+1])])
        else:
            ls.append([int(ps[i])+1, int(ps[i+1])])        
    return ls

def get_segment_features(x):
    ls = []
    for rg in x.positions:
        ls.append(x.feature[rg[0]:rg[1]])            
    return ls

def splitDataFrameList(df,target_column):
    def splitListToRows(row,row_accumulator,target_column):
        split_row = row[target_column]
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column))
    new_df = pd.DataFrame(new_rows)
    return new_df
                
def get_train_data():
    training_segment = pd.read_csv('training_segment.txt', header=None, names = ['segment'])
    training_segment['feature'] = joblib.load('train_feature.joblib')
    training_segment['positions'] = training_segment.apply(lambda x: get_segment_positions(x), axis=1)
    training_segment['feature'] = training_segment.apply(lambda x: get_segment_features(x), axis=1)
    training_segment = splitDataFrameList(training_segment, 'feature')
    training_segment['label'] = list(flatten(joblib.load('train_label.joblib')))
    training_segment = training_segment.drop(['segment','positions'], axis = 1)
    return training_segment

def get_test_data():
    test_segment = pd.read_csv('test_segment.txt', header=None, names = ['segment'])
    test_segment['feature'] = joblib.load('test_feature.joblib')
    test_segment['positions'] = test_segment.apply(lambda x: get_segment_positions(x), axis=1)
    test_segment['feature'] = test_segment.apply(lambda x: get_segment_features(x), axis=1)
    test_segment = splitDataFrameList(test_segment, 'feature')
    test_segment['ID'] = test_segment.index
    test_segment = test_segment.drop(['segment','positions'], axis = 1)
    return test_segment

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        ms = time.time()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    if is_inception and phase == 'train':
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)           

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)            
            print('{} Loss: {:.4f}, {} Acc: {:.4f}'.format(phase, epoch_loss, phase, epoch_acc))
        
        time_taken = str(datetime.timedelta(seconds=time.time() - ms))
        print('time taken: {}'.format(time_taken))
            

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [3]:
%%time
train_df = get_train_data()
print(train_df.info())
print(train_df.feature[0].shape)
print(train_df.feature[1])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7075 entries, 0 to 7074
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feature  7075 non-null   object
 1   label    7075 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 110.7+ KB
None
torch.Size([260, 400])
tensor([[-16.2783,   4.9215,  -3.6278,  ...,  -6.3865,  -1.0839,  -6.3627],
        [-19.2205,   6.1413,  -4.5396,  ...,  -5.2037,  -2.6973,  -4.9110],
        [-19.7272,   7.0195,  -4.0093,  ...,  -4.2315,  -7.2786,  -6.9802],
        ...,
        [-16.5469,   6.7402,  -2.2102,  ...,  -6.2332,  -6.7890,  -5.9929],
        [-18.1927,   5.1798,  -2.2626,  ...,  -5.0009,  -4.8182,  -4.7292],
        [-19.1877,   5.8449,  -4.2045,  ...,  -4.2093,  -6.4421,  -7.3169]],
       dtype=torch.float64)
CPU times: user 2.56 s, sys: 10.2 s, total: 12.7 s
Wall time: 12.8 s


In [4]:
%%time
def tensor_pad(x, max_len):
    target = torch.zeros([max_len, 400], dtype=torch.float64)
    w = x.shape[0]    
    if w <= max_len:       
        no_tile = (max_len // w) + 1        
        x = torch.cat(no_tile*[x])
    target = x[:max_len, :400]
    return target

def tensor_split(x, max_len):
    result = []    
    w = x.shape[0]
    if w <= max_len:
        result.append(tensor_pad(x, max_len))
    else:
        result = torch.split(x, max_len, dim=0)
        result = list(result)
        s =len(result)
        result[s-1] = tensor_pad(result[s-1], max_len)        
    
    for i in range(len(result)):
        result[i] = result[i].view(1, max_len, 400)
        
    return result

max_len = 400
train_data = train_df.sample(frac=1.0)
train_data['label'] = train_data.apply(lambda x : x["label"] - 1, axis = 1)
#train_data['feature'] = train_data.apply(lambda x : tensor_split(x.feature), axis = 1)
train_data['feature'] = train_data.apply(lambda x : tensor_split(x.feature, max_len), axis = 1)
train_data = splitDataFrameList(train_data, "feature")
print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10997 entries, 0 to 10996
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feature  10997 non-null  object
 1   label    10997 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 172.0+ KB
None
CPU times: user 40.2 s, sys: 7.43 s, total: 47.7 s
Wall time: 9.79 s


In [5]:
%%time
num_classes = 47
batch_size = 4

train_feature = torch.stack(train_data['feature'].tolist())
train_label = torch.tensor(train_data['label'].values.astype(np.long))

train_dataset = torch.utils.data.TensorDataset(train_feature, train_label)

train_size = int(0.8 * len(train_data))
val_size = int(0.1 * len(train_data))
test_size = len(train_data) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

dataloaders_dict = {'train':train_loader, 'val':val_loader}
print("len(train_loader.dataset) = ", len(train_loader.dataset))
print("len(val_loader.dataset) = ", len(val_loader.dataset))
print("len(test_loader.dataset) = ", len(test_loader.dataset))

len(train_loader.dataset) =  8797
len(val_loader.dataset) =  1099
len(test_loader.dataset) =  1101
CPU times: user 2.96 s, sys: 6.28 s, total: 9.24 s
Wall time: 9.25 s


In [ ]:
%%time
from torch.nn import *
from torch.nn.functional import *
from torchvision.models import *

class MyConv(Module):
    def __init__(self, in_channels, output_channels, kernel_size=3, stride=1, 
                 padding=0, activation_fn = relu, do_bn=True, with_bias = False):
        super(MyConv, self).__init__()  
        self.do_bn = do_bn
        self.activation_fn = activation_fn
        self.con = Conv2d(in_channels=in_channels, out_channels=output_channels, 
                          kernel_size=kernel_size, stride=stride, padding=padding,
                          bias=with_bias)
        if self.do_bn:
            self.bn = BatchNorm2d(num_features=output_channels, eps=0.001, momentum=0.01)
        
    def forward(self, x):   
        x = self.con(x)        
        if self.do_bn:
            x = self.bn(x)
        if self.activation_fn:
            x = self.activation_fn(x)
        return x
    
class MyInception(Module):
    def __init__(self, in_channels, out_channels):
        super(MyInception, self).__init__()
        self.conv_0 = MyConv(in_channels=in_channels, output_channels=out_channels[0], kernel_size=3)
        self.conv_1 = MyConv(in_channels=in_channels, output_channels=out_channels[1], kernel_size=1)
        self.conv_2 = MyConv(in_channels=out_channels[1], output_channels=out_channels[2], kernel_size=3)
        self.conv_3 = MyConv(in_channels=in_channels, output_channels=out_channels[3], kernel_size=1)
        self.conv_4 = MyConv(in_channels=out_channels[3], output_channels=out_channels[4], kernel_size=3)
        self.pool = MaxPool2d(kernel_size=3, stride=1, padding=0)
        self.conv_5 = MyConv(in_channels=in_channels, output_channels=out_channels[5], kernel_size=1)

    def forward(self, x):    
        r_0 = self.conv_0(x)
        r_1 = self.conv_2(self.conv_1(x))
        r_2 = self.conv_4(self.conv_3(x))
        r_3 = self.conv_5(self.pool(x))
        #print(r_0.shape, r_1.shape, r_2.shape, r_3.shape)
        return torch.cat([r_0,r_1,r_2,r_3], dim=1)

class MyNet(Module):
    def __init__(self, batch_size, num_classes, verbose = False):
        super(MyNet, self).__init__()
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.verbose = verbose
        
        self.l1 = MyConv(in_channels=1, output_channels=64, kernel_size=7, stride=2, padding=3)
        self.l2 = MaxPool2d(kernel_size=3, stride=2)
        self.l3 = MyConv(in_channels=64, output_channels=64, kernel_size=1)
        self.l4 = MyConv(in_channels=64, output_channels=192, kernel_size=3, padding=1)
        self.l5 = MaxPool2d(kernel_size=3, stride=3)
        self.l6 = MyInception(192, [64,96,128,16,32,32])
        self.l7 = MyInception(256, [128,128,192,32,96,64])
        self.l8 = MaxPool2d(kernel_size=3, stride=2)
        self.l9 = MyInception(128+192+96+64, [192,96,208,16,48,64])
        self.l10 = MyInception(192+208+48+64, [160,112,224,24,64,64])
        self.l11 = MyInception(160+224+64+64, [128,128,256,24,64,64])
        self.l12 = MyInception(128+256+64+64, [112,144,288,32,64,64])
        self.l13 = MyInception(112+288+64+64, [256,160,320,32,128,128])
        #self.l14 = MaxPool2d(kernel_size=1, stride=1)
        #self.l15 = MyInception(256+320+128+128, [256,160,320,32,128,128])
        #self.l16 = MyInception(256+320+128+128, [384,192,384,48,128,128]) 
        self.l17 = AvgPool2d(kernel_size=2, stride=1)
        self.l18 = Dropout(p=0.5)
        #self.l19 = MyConv(in_channels=384+384+128+128, output_channels=num_classes, kernel_size=1, activation_fn=None, do_bn=False, with_bias=True)
        self.l19 = MyConv(in_channels=256+320+128+128, output_channels=num_classes, kernel_size=1, activation_fn=None, do_bn=False, with_bias=True)
        self.l20 = Linear(in_features=423, out_features=num_classes)
    
    def forward(self, x):
        
        x = self.l1(x)
        if self.verbose:
            print("self.l1(x)", x.shape)
        x = self.l2(x)
        if self.verbose:
            print("self.l2(x)", x.shape)
        x = self.l3(x)
        if self.verbose:
            print("self.l3(x)", x.shape)
        x = self.l4(x)
        if self.verbose:
            print("self.l4(x)", x.shape)
        x = self.l5(x)
        if self.verbose:
            print("self.l5(x)", x.shape)
        x = self.l6(x)
        if self.verbose:
            print("self.l6(x)", x.shape)
        x = self.l7(x)
        if self.verbose:
            print("self.l7(x)", x.shape)
        x = self.l8(x)
        if self.verbose:
            print("self.l8(x)", x.shape)
        x = self.l9(x)
        if self.verbose:
            print("self.l9(x)", x.shape)
        x = self.l10(x)
        if self.verbose:
            print("self.l10(x)", x.shape)
        x = self.l11(x)
        if self.verbose:
            print("self.l11(x)", x.shape)
        x = self.l12(x)
        if self.verbose:
            print("self.l12(x)", x.shape)
        x = self.l13(x)
        if self.verbose:
            print("self.l13(x)", x.shape)
        #x = self.l14(x)
        #if self.verbose:
        #    print("self.l14(x)", x.shape)
        #x = self.l15(x)
        #if self.verbose:
        #    print("self.l15(x)", x.shape)
        #x = self.l16(x)
        #if self.verbose:
        #    print("self.l16(x)", x.shape)
        x = self.l17(x)
        if self.verbose:
            print("self.l17(x)", x.shape)
        x = self.l18(x)
        if self.verbose:
            print("self.l18(x)", x.shape)
        x = self.l19(x)
        if self.verbose:
            print("self.l19(x)", x.shape)        
        #print("x.view(4, -1)", x.view(4, -1).shape)
        x = x.view(-1, 423)        
        x = softmax(self.l20(x))        
        return x

num_epochs = 5
feature_extract = True

model_ft = MyNet(batch_size, num_classes).double()
print(model_ft)
model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

optimizer_ft = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)  
criterion = torch.nn.CrossEntropyLoss()
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=False)

MyNet(
  (l1): MyConv(
    (con): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  )
  (l2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (l3): MyConv(
    (con): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  )
  (l4): MyConv(
    (con): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  )
  (l5): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (l6): MyInception(
    (conv_0): MyConv(
      (con): Conv2d(192, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    )
    (conv_1): MyConv(
  

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


train Loss: 3.7649, train Acc: 0.1275
val Loss: 3.6982, val Acc: 0.1865
time taken: 0:10:05.905303
Epoch 1/4
----------
train Loss: 3.7043, train Acc: 0.1814
val Loss: 3.7078, val Acc: 0.1793
time taken: 0:10:05.298587
Epoch 2/4
----------
train Loss: 3.6687, train Acc: 0.2180
val Loss: 3.6468, val Acc: 0.2384
time taken: 0:10:05.005693
Epoch 3/4
----------
train Loss: 3.6566, train Acc: 0.2293
val Loss: 3.6435, val Acc: 0.2429
time taken: 0:10:49.498328
Epoch 4/4
----------


In [ ]:
%%time
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model_ft(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of test data: {0}'.format(100 * correct / total))

In [ ]:
%%time
model_name = "model_3.2.model" 
torch.save(model_ft.state_dict(), model_name)
upload_files([model_name])

In [ ]:
%%time
batch_size = 4
num_classes = 47

model_name = "model_3.2.model"
model = MyNet(batch_size, num_classes).double().to(device)
model.load_state_dict(torch.load(model_name, map_location=device)) 

In [ ]:
%%time
test_df = get_test_data()
print(test_df.info())

In [ ]:
%%time
test_data = test_df.sample(frac = 1.0)
test_data['feature'] = test_data.apply(lambda x : tensor_split(x.feature), axis = 1)
test_data = splitDataFrameList(test_data, "feature")
print(test_data.info())

test_feature = torch.stack(test_data['feature'].tolist())
test_dataset = torch.utils.data.TensorDataset(test_feature)
predict_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
result = []
with torch.no_grad():
    for data in predict_loader:
        inputs = data[0].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        result = np.append(result, predicted.cpu())


In [ ]:
%%time
test_data['label'] = np.transpose(result).astype('int') 
test_data_group = test_data.groupby('ID')['label'].apply(list).reset_index(name='labels')
test_data_group['Category'] = test_data_group.apply(lambda x : np.argmax(np.bincount(x.labels)) + 1, axis = 1)
test_data_group = test_data_group[['ID','Category']]
test_data_group

In [ ]:
len(test_data_group.Category.unique())

In [ ]:
submission_name = "submission_3.2_max.csv"
test_data_group.to_csv(submission_name, index=False)

In [ ]:
upload_files([submission_name])

In [ ]:
%%time
test_df = get_test_data()
print(test_df.info())

In [ ]:
%%time
test_data = test_df.sample(frac = 1.0)
test_data['feature'] = test_data.apply(lambda x : tensor_split(x.feature), axis = 1)
test_data = splitDataFrameList(test_data, "feature")
print(test_data.info())

test_feature = torch.stack(test_data['feature'].tolist())
test_dataset = torch.utils.data.TensorDataset(test_feature)
predict_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)
result = []
with torch.no_grad():
    for data in predict_loader:
        inputs = data[0]            
        inputs = inputs.to(device)    
        outputs = model(inputs)  
        prop = torch.nn.functional.softmax(outputs).cpu().tolist()
        result = np.append(result, prop)

In [ ]:
%%time
test_result = pd.DataFrame(result.reshape(-1,47))
test_result['ID'] = test_data['ID'].values
test_data_mean = test_result.groupby('ID')[list(range(47))].agg(['mean'])
test_data_mean.columns = list(range(47))
test_data_mean["Category"] = test_data_mean.idxmax(axis = 1)
test_data_mean['Category'] = test_data_mean.apply(lambda x : x["Category"] + 1, axis = 1).astype('int') 
test_data_mean["ID"] = test_data_mean.index
test_data_group = test_data_mean[['ID','Category']]
print(test_data_group.info())

In [ ]:
len(test_data_group.Category.unique())

In [ ]:
submission_name = "submission_3.2_mean.csv"
test_data_group.to_csv(submission_name, index=False)

In [ ]:
upload_files([submission_name])